In [506]:
import cv2 as cv
import os
from argparse import ArgumentParser


In [498]:
# read imaged
folder = 'Bibby_1931-p4'
if not os.path.exists(folder):
    os.mkdir(folder)
filename = 'Bibby_1931-p4.jpg'
img = cv.imread('../testing/' + filename)

In [499]:
# convert image to grey
grey_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY )
cv.imwrite(folder + "/01-greyscale.jpg", grey_image)

True

In [500]:
# dilate and write into grey_image
kernel = cv.getStructuringElement(cv.MORPH_RECT,(15,15))
cv.dilate(grey_image, kernel, grey_image)
cv.imwrite(folder + "/02-dilated.jpg", grey_image)

True

In [501]:
# change white to black and invert
ret1,dst = cv.threshold(grey_image, 240, 255, cv.THRESH_TOZERO)
ret1,dst = cv.threshold(dst, 240, 255, cv.THRESH_BINARY_INV)
cv.imwrite(folder + "/03-inverted.jpg", dst)

True

In [502]:
#try to fill images rectangles and remove noise
kernel = cv.getStructuringElement(cv.MORPH_RECT,(20,20))

gradient = cv2.morphologyEx(dst, cv2.MORPH_CLOSE, kernel)
grad_img = cv2.morphologyEx(gradient, cv2.MORPH_OPEN, kernel)
cv.imwrite(folder + "/04-morphed.jpg", grad_img)

True

In [503]:
# find contours
contours, hierachy = cv.findContours(grad_img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

In [504]:
# draw contours with rectangle
img_with_cont = grad_img

for j in range(len(contours)):
    area = cv.contourArea(contours[j])
    print(area)
    if area > 5000:
        x,y,w,h = cv.boundingRect(contours[j])
        cropped = img[y:y+h, x:x+w]
        cv.imwrite(folder + "/07-copped-bb-" + str(j) + ".jpg", cropped)
        cv.rectangle(img_with_cont, (x, y), (x + w, y + h), (161,155,86), 5)
        #img_with_cont = cv.drawContours(img_with_cont, contours, j, [161,155,86], thickness=5)
cv.imwrite(folder + "/05-img_with_cont.jpg", img_with_cont)

380.0
361.0
337376.0
362873.5
665.0
140503.5
197972.0
400.0
1007.0
494.0
828751.0


True

In [505]:
# draw contours with poly
img_with_cont = grad_img

for j in range(len(contours)):
    cnt = contours[j]
    area = cv.contourArea(cnt)
    if area > 5000:
        epsilon = 0.15*cv.arcLength(cnt,True)
        approx = cv.approxPolyDP(cnt,epsilon,True)
        x,y,w,h = cv.boundingRect(approx)
        cropped = img[y:y+h, x:x+w]
        cv.imwrite(folder + "/07-copped-" + str(j) + ".jpg", cropped)
        
        img_with_cont = cv.drawContours(img_with_cont, [approx],0, [161,155,86], thickness=5)
cv.imwrite(folder + "/06-img_with_approx.jpg", img_with_cont)

True